# HW1.1: Dictionary-based Tokenization


In this exercise, you are to implement a dictionary-based word segmentation algorithm. There are two Python functions that you need to complete:
<br>
* maximal_matching
* backtrack
</br>

Also, you have to find how to use word_tokenize() in PythaiNLP along with customer_dict by yourselves.

In [1]:
# !pip install pythainlp
# !pip install marisa_trie
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import get_corpus
from marisa_trie import Trie

## Part 1) Maximal Matching from PythaiNLP

### Create a toy dictionary to test the algorithm

This is based on the example shown in the lecture.
You will tokenize the following text string: "ไปหามเหสี!"
The toy dictionary provided in this exercise includes all the charaters, syllables, and words that appear that the text string.

In [2]:
thai_vocab = ["ไ","ป","ห","า","ม","เ","ห","ส","ี","ไป","หา","หาม","เห","สี","มเหสี","!"]
input_text = "ไปหามเหสี!"

### Example Dictionary

Write the `word_tokenize` function of PyThaiNLP with a custom dictionary above and using:
1. Longest matching algorithm `longest`
2. Maximal-matching algorithm `newmm`

Study `word_tokenize()` from PythaiNLP in the link below. Note: `custom_dict` will accept Trie structures as `Trie(iterable)`.

https://pythainlp.org/docs/5.0/api/tokenize.html#pythainlp.tokenize.word_tokenize

In [3]:
####FILL CODE HERE####
# Tokenize using 'longest' algorithm
tokens_longest = word_tokenize(input_text, custom_dict=Trie(thai_vocab), engine='longest')
print("Longest:", tokens_longest)

# Tokenize using 'newmm' algorithm
tokens_newmm = word_tokenize(input_text, custom_dict=Trie(thai_vocab), engine='newmm')
print("Maximal-matching:", tokens_newmm)
######################

Longest: ['ไป', 'หาม', 'เห', 'สี', '!']
Maximal-matching: ['ไป', 'หา', 'มเหสี', '!']


## Part 2) Maximal Matching from Scratch

### Maximal matching
Complete the maximal matching function below with dynamic programming to tokenize the input text and output the 2D numerical array shown in class.

In [4]:
# from math import inf #infinity
import numpy as np

def maximal_matching(c, dictionary=Trie(thai_vocab)):
    #Initialize an empty 2D list
    n = len(c)
    # d = [[None]*n for _ in range(n)]
    d = np.full((n, n), None)
    ####FILL CODE HERE####
    for i in range(n):
        for j in range(i, n):
            if i==0 and c[0:j+1] in dictionary:
                d[i][j] = 1
            elif c[i:j+1] in dictionary:
                d[i][j] = 1 + np.min(d[:i,i-1])
            else:
                d[i][j] = np.inf
    ######################

    return d

### Test your maximal matching algorithm on a toy dictionary

Expected output:

[1, 1, inf, inf, inf, inf, inf, inf, inf, inf] ไ
<br>
[None, 2, inf, inf, inf, inf, inf, inf, inf, inf] ป
<br>
[None, None, 2, 2, 2, inf, inf, inf, inf, inf] ห
<br>
[None, None, None, 3, inf, inf, inf, inf, inf, inf] า
<br>
[None, None, None, None, 3, inf, inf, inf, 3, inf] ม
<br>
[None, None, None, None, None, 3, 3, inf, inf, inf] เ
<br>
[None, None, None, None, None, None, 4, inf, inf, inf] ห
<br>
[None, None, None, None, None, None, None, 4, 4, inf] ส
<br>
[None, None, None, None, None, None, None, None, 5, inf] ี
<br>
[None, None, None, None, None, None, None, None, None, 4] !
<br>

In [5]:
input_text = "ไปหามเหสี!"
out = maximal_matching(input_text)
for i in range(len(out)):
    print(out[i],input_text[i])

[1 1 inf inf inf inf inf inf inf inf] ไ
[None 2 inf inf inf inf inf inf inf inf] ป
[None None 2 2 2 inf inf inf inf inf] ห
[None None None 3 inf inf inf inf inf inf] า
[None None None None 3 inf inf inf 3 inf] ม
[None None None None None 3 3 inf inf inf] เ
[None None None None None None 4 inf inf inf] ห
[None None None None None None None 4 4 inf] ส
[None None None None None None None None 5 inf] ี
[None None None None None None None None None 4] !


### Backtracking
Complete the backtracking function below to find the tokenzied words.
It should return a list containing a pair of the beginning position and the ending position of each word.
In this example, it should return:
<br>
[(0, 1),(2, 3),(4, 8),(9, 9)]
<br>
#### Each pair contains the position of each word as follows:
(0, 1) ไป
<br>
(2, 3) หา
<br>
(4, 8) มเหสี
<br>
(9, 9) !


In [6]:
def backtrack(d):
    eow = len(d)-1 # End of Word position
    word_pos = [] # Word position # list of pairs

    ####FILL CODE HERE####
    d = np.where(d==None, np.inf, d) # replace None with INF
    while eow >= 0:
        bow = np.argmin(d[:,eow], axis=0) # bow = beginning of word
        word_pos.append((bow, eow))
        eow = bow - 1
    ######################
    word_pos.reverse()
    return word_pos

backtrack(out)

[(0, 1), (2, 3), (4, 8), (9, 9)]

### Test your backtracking algorithm on a toy dictionary
Compare your results with the result from PyThaiNLP `newmm`.

Expected output:
<br>
ไป|หา|มเหสี|!

In [7]:
def print_tokenized_text(d, input_text):
    tokenized_text=[]
    for pos in backtrack(d):
        #print(pos)
        tokenized_text.append(input_text[pos[0]:pos[1]+1])
    print(len(tokenized_text))
    print("|".join(tokenized_text))

print_tokenized_text(out,input_text)

4
ไป|หา|มเหสี|!


### <font color=blue>Question 1</font>
Using your maximal matching code with the toy dictionary, how many “words” did you get when tokenizing this input text.

Answer this question in question #1 in MyCourseVille. Also print out the answer in this notebook as well.

In [8]:
input_text = "ไปหาหมามเหสีมาหาม!"

print_tokenized_text(maximal_matching(input_text),input_text)

10
ไป|หา|ห|ม|า|มเหสี|ม|า|หาม|!


## Part 3) Your Maximal Matching with Real Dictionary

For UNIX-based OS users, the following cell will download a dictionary (it's just a list of thai words). Alternatively, you can download it from this link: https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt

In [9]:
# !wget https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt
# !curl -O https://raw.githubusercontent.com/PyThaiNLP/pythainlp/dev/pythainlp/corpus/words_th.txt

In [10]:
with open("words_th.txt",encoding='utf-8-sig') as f:
    thai_vocab = f.read().splitlines()
print("Vocab size:", len(thai_vocab))
print(thai_vocab[:10])

thai_vocab.extend(["ๆ","!"])

Vocab size: 62077
['ก ข ไม่กระดิกหู', 'ก.', 'ก.ค.', 'ก.ต.', 'ก.ป.ส.', 'ก.พ.', 'ก.พ.ด.', 'ก.ม.', 'ก.ย.', 'ก.ย']


### Part 3.1) The output of **YOUR** maximal matching algoithm on the new dictionary
Expected output:
<br>
[inf, 1, inf, 1, inf, inf, inf, inf, inf] ไ
<br>
[None, inf, inf, inf, inf, inf, inf, inf, inf] ป
<br>
[None, None, inf, 2, 2, inf, inf, inf, inf] ห
<br>
[None, None, None, inf, inf, inf, inf, inf, inf] า
<br>
[None, None, None, None, inf, inf, inf, inf, 2] ม
<br>
[None, None, None, None, None, inf, 3, inf, inf] เ
<br>
[None, None, None, None, None, None, inf, inf, inf] ห
<br>
[None, None, None, None, None, None, None, inf, 4] ส
<br>
[None, None, None, None, None, None, None, None, inf] ี

### Expected tokenized text
ไปหา|มเหสี

_Question: Why are the resulting tokens different?_

In [11]:
input_text = "ไปหามเหสี"
out = maximal_matching(input_text, dictionary=Trie(thai_vocab))
for i in range(len(out)):
    print(out[i],input_text[i])

print_tokenized_text(out,input_text)

[inf 1 inf 1 inf inf inf inf inf] ไ
[None inf inf inf inf inf inf inf inf] ป
[None None inf 2 2 inf inf inf inf] ห
[None None None inf inf inf inf inf inf] า
[None None None None inf inf inf inf 2] ม
[None None None None None inf 3 inf inf] เ
[None None None None None None inf inf inf] ห
[None None None None None None None inf 4] ส
[None None None None None None None None inf] ี
2
ไปหา|มเหสี


### <font color=blue>Question 2</font>
Using your maximal matching algorithm and the actual Thai dictionary, how many “words” did you get when tokenizing this input text.

Answer this question in question #2 in MyCourseVille. Also print out the answer in this notebook as well.

In [12]:
input_text = "ประเทศไทยรวมเลือดเนื้อชาติเชื้อไทยเป็นประชารัฐไผทของไทยทุกส่วนอยู่ดำรงคงไว้ได้ทั้งมวลด้วยไทยล้วนหมายรักสามัคคี"

out = maximal_matching(input_text, dictionary=Trie(thai_vocab))
print_tokenized_text(out,input_text)

26
ประเทศ|ไทย|รวม|เลือดเนื้อ|ชาติ|เชื้อ|ไทย|เป็น|ประชา|รัฐ|ไผท|ของ|ไทย|ทุก|ส่วน|อยู่|ดำรง|คงไว้|ได้|ทั้งมวล|ด้วย|ไทย|ล้วน|หมาย|รัก|สามัคคี


### Part 3.2) Use PyThaiNLP `word_tokenize` with custom dictionary

Try tokenizing the following text with `word_tokenize` in `newmm` algorithm and default real dictionary.

In [13]:
text='นัดกินกันตอนไหนก็ได้ที่สามย่านมิตรทาวน์'

####FILL CODE HERE####
word_tokenize(text, custom_dict=Trie(thai_vocab), engine='newmm')
######################

['นัด', 'กินกัน', 'ตอน', 'ไหน', 'ก็', 'ได้ที่', 'สามย่าน', 'มิตร', 'ทาวน์']

Add 'สามย่านมิตรทาวน์' into dictionary and then tokenize again

In [14]:
####FILL CODE HERE####
thai_vocab.append("สามย่านมิตรทาวน์")
word_tokenize(text, custom_dict=Trie(thai_vocab), engine='newmm')
######################

['นัด', 'กินกัน', 'ตอน', 'ไหน', 'ก็', 'ได้ที่', 'สามย่านมิตรทาวน์']

### <font color=blue>Question 3</font>
Using the code from part three only, how many “words” did you get when tokenizing this input text **after adding the new vocabs**.

Answer this question in question #3 in MyCourseVille. Also print out the answer in this notebook as well.

In [15]:
new_vocab = ["ดิสนีย์ออนไอซ์", "ตีกอล์ฟ", "ธรรมมะ"]
input_text = "อ๋อก็ว่าจะไปเรียนแต่งหน้านั่งสมาธิดำน้ำปลูกปะการังทำอาหารนวดสปาปลูกป่าดำนาดูดิสนีย์ออนไอซ์แรลลี่ตีกอล์ฟล่องเรือส่องสัตว์ช้อปปิ้งดูงิ้วดูละครเวทีดูคอนเสิร์ตดินเนอร์ทำขนมจัดดอกไม้เที่ยวตลาดน้ำเรียนถ่ายรูปดูกายกรรมชมเมืองเก่าเข้าสัมมนาทัวร์ธรรมมะเรียนเต้นแล้วก็ร้องเพลง"

thai_vocab.extend(new_vocab)
out = maximal_matching(input_text, dictionary=Trie(thai_vocab))
print_tokenized_text(out,input_text)

51
อ๋อ|ก็|ว่า|จะ|ไป|เรียน|แต่งหน้า|นั่งสมาธิ|ดำน้ำ|ปลูก|ปะการัง|ทำอาหาร|นวด|สปา|ปลูกป่า|ดำนา|ดู|ดิสนีย์ออนไอซ์|แรลลี่|ตีกอล์ฟ|ล่องเรือ|ส่องสัตว์|ช้อปปิ้ง|ดู|งิ้ว|ดู|ละครเวที|ดู|คอนเสิร์ต|ดินเนอร์|ทำ|ขนม|จัด|ดอกไม้|เที่ยว|ตลาดน้ำ|เรียน|ถ่ายรูป|ดู|กายกรรม|ชม|เมือง|เก่า|เข้า|สัมมนา|ทัวร์|ธรรมมะ|เรียน|เต้น|แล้วก็|ร้องเพลง


## Part 4) Use maximal matching on real dataset

To complete this exercise, we will use the maximal matching algorithm on NECTEC's BEST corpus.

The corpus has a structure of characters with target whether it's a beginning of a word (True/False).

In [16]:
#Download dataset
# !gdown "1EcrlXYUyIEM3aeIJse6nPpiv_UjSKgoU&confirm=t"

In [17]:
# !tar xvf corpora.tar.gz

In [18]:
import pandas as pd
import os

In [19]:
# Path to the preprocessed data
best_processed_path = 'corpora/BEST'
option = "test"

df = []
# article types in BEST corpus
article_types = ['article', 'encyclopedia', 'news', 'novel']
for article_type in article_types:
    df.append(pd.read_csv(os.path.join(best_processed_path, option, 'df_best_{}_{}.csv'.format(article_type, option))))
df = pd.concat(df)
df

,char,target
0,ป,True
1,ฏ,False
2,ิ,False
3,ร,False
4,ู,False
...,...,...
644911,ห,False
644912,น,False
644913,ม,True
644914,า,False


In [20]:
len(df)

2271932

In [21]:
# Some text in this corpus
all_text = "".join(df['char'].tolist())
all_text[:1000]

'ปฏิรูปการศึกษา : มุมมองทางกระบวนทัศน์และบริบทสังคมไทยThe Reformation of Eucation from A Thai Perspectiveกระบวนทัศน์และวิธีคิดแบบแยกส่วน ลดส่วน ได้ทำให้"การศึกษาเรียนรู้"ใน หลายทศวรรษที่ผ่านมา กลายเป็นเรื่องของนักวิชาการด้านศึกษาศาสตร์ ครุศาสตร์ หรือเป็นเรื่องของโรงเรียน ครูอาจารย์ กระทรวงศึกษาธิการ ทบวงมหาวิทยาลัยฯ มาอย่างต่อเนื่องยาวนาน (เหมือนกับที่เรื่องสุขภาพเป็นเรื่องของแพทย์และโรงพยาบาล) การจัดการศึกษาภายใต้กระบวนทัศน์และวิธีคิดแบบดังกล่าวของรัฐ ได้ถูกวิพากษ์วิจารณ์และตกเป็นจำเลยจากวิกฤตการณ์ทางสังคมมากมาย อันสะท้อนถึงความล้มเหลวของการจัดการศึกษาเพื่อพัฒนามนุษย์ (ปัญหาศีลธรรมเสื่อมถอย ยาเสพติด การขาดจิตสำนึกทางสังคม ฯลฯ) ซึ่งสังคมร่วมกันสรุปว่า เกิดจากความล้มเหลวของระบบการศึกษาในกระบวนทัศน์แบบแยกส่วน นำมาสู่การปฏิรูปการศึกษาที่กำลังดำเนินการอยู่ในปัจจุบัน ด้วยเป้าหมายเพื่อสร้างการเรียนรู้แบบองค์รวม ที่จะทำให้"ผู้เรียนเก่ง-ดี-มีความสุข"คำถามที่ผู้เขียนสนใจในการปฏิรูปการศึกษาที่ดำเนินการในปัจจุบัน คือ๑.การปฏิรูปการศึกษาในปัจจุบันดำเนินการภายใต้กระบวนทัศน์แบบบูรณาการ (องค์รวม)ตามที

### <font color=blue>Question 4</font>
Using PyThaiNLP `newmm`, how many words did you get in the BEST corpus (test)? [Runtime is around 7 mins] What are the accuracy, f1, precision, recall scores for each character?

Answer this question in question #4 in MyCourseVille. Also print out the answer in this notebook as well.

_Question: What main metric should we look at? Why?_

In [22]:
####FILL CODE HERE####
tokens = word_tokenize(text=all_text, engine='newmm')

# save
import pickle
with open('tokens.pkl', 'wb') as f:
    pickle.dump(tokens, f)
######################

In [26]:
import pickle
# load
with open('tokens.pkl', 'rb') as f:
    tokens = pickle.load(f)

In [29]:
len(tokens)

569631

In [23]:
def convert_to_character(_tokens):
  char_list = [0]*len("".join(_tokens))
  char_count = 0
  for word in _tokens:
    char_list[char_count] = 1
    char_count += len(word)
  return char_list

chars = convert_to_character(tokens)
chars[:20]

[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]

In [37]:
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

####FILL CODE HERE####
accuracy = accuracy_score(df['target'], chars)
f1 = f1_score(df['target'], chars)
precision = precision_score(df['target'], chars)
recall = recall_score(df['target'], chars)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
######################

Accuracy: 0.94
F1: 0.89
Precision: 0.94
Recall: 0.85
